In [28]:
import numpy as np 

In [29]:
class Data():
    """Process and load data
    """
    def __init__(self, filename: str = 'mc_ggH_16_13TeV_Zee_EGAM1_calocells_16249871.csv') -> None:
        """setting initial data specific parameters.

        Args:
            filename (str, optional): data file to load. Defaults to 'mc_ggH_16_13TeV_Zee_EGAM1_calocells_16249871.csv'.
        """
        self.data = self.read_data(filename)
        self.nevents = self.data.shape[0]
        self.name = ["averageInteractionsPerCrossing", "p_Rhad","p_Rhad1", "p_TRTTrackOccupancy", "p_topoetcone40", 
                     "p_eTileGap3Cluster", "p_phiModCalo", "p_etaModCalo"]
        self.NvtxReco = self.data[:,1]
        self.p_nTracks = self.data[:,2]
        self.p_truthType = self.data[:,10]

        self.signal = self.p_truthType == 2

        self.data = self.data[:, [0,3,4,5,6,7,8,9]]

        self.means_sig = np.array([np.average(self.data[self.signal, i]) for i in range(8)])
        self.means_bckg = np.array([np.average(self.data[~self.signal, i]) for i in range(8)])
        self.flip = np.sign(self.means_bckg - self.means_sig)
            
        for i in range(8): 
            self.data[:, i] *= self.flip[i]
            self.means_sig[i]  = self.means_sig[i] * self.flip[i]
            self.means_bckg[i] = self.means_bckg[i] * self.flip[i]


    
    def read_data(self, filename = 'mc_ggH_16_13TeV_Zee_EGAM1_calocells_16249871.csv'):
        return np.loadtxt(filename, delimiter = ',', skiprows = 1, usecols=range(1,12))


In [30]:
data = Data()

In [33]:
data.means_sig

array([ 3.89248969e+01,  6.77550670e-03,  5.07774657e-03,  4.18748975e-01,
        1.95529984e+00, -8.67498142e-01, -1.10297362e-05,  1.24595979e-02])

In [ ]:
# ikke færdig overhovedet...
def master(nworker: int, data: np.array):
    ranges = np.zeros([n_cuts, 8])
    
    # loop over different event channels and set up cuts
    for i in range(8):
        for j in range n_cuts:
            ranges[j][i] = data.means_sig[i] + j * (data.means_bckg[i] - data.means_sig[i]) / n_cuts
    
    print("Best accuracy obtained:", best_accuracy_score, "\n")
    print("Final cuts: \n")
    
    
    print()
    print("Number of settings:", n_settings, "\n")
    print("Elapsed time:", (tend - tstart), "\n")
    print("task time [mus]:", (tend - tstart)/ n_settings, "\n")

In [ ]:
def task_function(setting: list, data: Data) -> float:
    pred = np.min(data.data < setting, axis=0)
    return np.sum(pred == data.signal) / len(data.data)